In [3]:
import climdata
from hydra import initialize, compose
import xclim
import datetime

In [ ]:
mirror = climdata.ERA5(base_path="./zarr_tas_data")

# split the years into train, validation, and test
years = list(range(2025, 2025+1))
# Set the variables to download for 34 var dataset
date_range = (
    datetime.date(2000, 1, 1),
    datetime.date(2000, 1, 3),
)

mirror.download("2m_temperature",date_range)

In [ ]:
with initialize(config_path="../climdata/conf", version_base=None):
    cfg = compose(config_name="config",
                  overrides=[
                      "dataset=mswx",
                      "weather.parameter=tasmax",
                      "time_range.start_date=2000-06-01",
                      "time_range.end_date=2000-06-30",
                      ])
cfg.mappings[cfg.dataset].params['google_service_account'] = "/home/muduchuru/.climdata_conf/service.json"

In [14]:
mswx_ob = climdata.MSWX(cfg)
files = mswx_ob.fetch()
dset = mswx_ob.load()
dset_subset = mswx_ob.extract(point=(52,20))
dset_subset = dset_subset.compute()
dset_subset = xclim.core.units.convert_units_to(dset_subset,cfg.mappings['info'][cfg.weather.parameter].units)

df = mswx_ob.to_dataframe(dset_subset)
df = mswx_ob.format(df)

📂 0 exist, 30 missing — fetching from Drive...
⬇️ Downloading 2025181.nc ...
   → Download 100% complete
⬇️ Downloading 2025180.nc ...
   → Download 100% complete
⬇️ Downloading 2025179.nc ...
   → Download 100% complete
⬇️ Downloading 2025178.nc ...
   → Download 100% complete
⬇️ Downloading 2025177.nc ...
   → Download 100% complete
⬇️ Downloading 2025176.nc ...
   → Download 100% complete
⬇️ Downloading 2025175.nc ...
   → Download 100% complete
⬇️ Downloading 2025174.nc ...
   → Download 100% complete
⬇️ Downloading 2025173.nc ...
   → Download 100% complete
⬇️ Downloading 2025172.nc ...
   → Download 100% complete
⬇️ Downloading 2025171.nc ...
   → Download 100% complete
⬇️ Downloading 2025170.nc ...
   → Download 100% complete
⬇️ Downloading 2025169.nc ...
   → Download 100% complete
⬇️ Downloading 2025168.nc ...
   → Download 100% complete
⬇️ Downloading 2025167.nc ...
   → Download 100% complete
⬇️ Downloading 2025166.nc ...
   → Download 100% complete
⬇️ Downloading 2025165.nc

In [ ]:
# initialize
mswx_ob = climdata.MSWX(cfg)

# fetch + load
files = mswx_ob.fetch()
dset = mswx_ob.load()

dset_subset = mswx_ob.extract(box=(
    14.0, 49.0, 24.0, 55.0
    ))
dset_subset = dset_subset.compute()

# convert to dataframe
df = mswx_ob.to_dataframe(dset_subset)

# format dataframe
formatted = mswx_ob.format(df)

In [ ]:
formatted

In [ ]:
import argparse
import climdata
from hydra import initialize, compose
import pandas as pd
from climdata import CMIP
import xclim

with initialize(config_path="../climdata/conf", version_base=None):
    cfg = compose(
        config_name="config",
        overrides=[
            f"weather.parameter=pr",  # Just for bounds/region
            f"region=europe",
            f"time_range.start_date=2000-08-01",
            f"time_range.end_date=2000-08-31",
        ],
    )

cmip = climdata.CMIP(
    experiment_id = "historical",
    source_id = "EC-Earth3-Veg",
    table_id = "day",
    variables = ["tasmax", "tasmin", "pr"],
    region_bounds = cfg.bounds[cfg.region]
    )
cmip.fetch()      # gets file lists
cmip.load()       # loads and merges datasets
cmip.extract(point = (52,15))
ds = cmip._subset_time(cfg.time_range.start_date, cfg.time_range.end_date)
for var in ds.data_vars:
    ds[var] =  xclim.core.units.convert_units_to(
        ds[var], cfg.mappings["info"][var].units
    )
cmip.save_csv("cmip_loc.csv")

[<pangeo-cmip6 catalog with 1 dataset(s) from 50 asset(s)>,
 <pangeo-cmip6 catalog with 1 dataset(s) from 50 asset(s)>,
 <pangeo-cmip6 catalog with 1 dataset(s) from 50 asset(s)>]

In [4]:
cmip.col.df[cmip.col.df["experiment_id"]=="historical"]

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
514,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,AERmon,emidust,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180301
515,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,AERmon,emiss,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180301
516,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,AERmon,mmrbc,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180301
517,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,AERmon,mmrdust,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180301
518,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,AERmon,mmroa,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180301
...,...,...,...,...,...,...,...,...,...,...,...
514813,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514814,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tauu,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514815,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hur,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207
514816,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hus,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20211207


In [3]:
import argparse
import climdata
from hydra import initialize, compose
import pandas as pd
from climdata import CMIP

region="europe"

with initialize(config_path="../climdata/conf", version_base=None):
    cfg = compose(
        config_name="config",
        overrides=[
            f"region={region}",
            f"time_range.start_date=2000-08-01",
            f"time_range.end_date=2000-08-31",
        ],
    )

cmip = climdata.CMIP(
    experiment_id = "historical",
    source_id = "MIROC6",
    table_id = "day",
    variables = ["tasmax", "tasmin", "pr"],
    region_bounds = cfg.bounds[cfg.region]
    )
cmip.fetch()      # gets file lists
cmip.load()       # loads and merges datasets

cmip.extract(box = cmip.region_bounds)
ds = cmip._subset_time(cfg.time_range.start_date, cfg.time_range.end_date)
cmip.save_netcdf("cmip_box.nc")

Saved NetCDF to cmip_box.nc


In [4]:
ds

<xarray.Dataset> Size: 334kB
Dimensions:    (lat: 27, bnds: 2, lon: 33, time: 31)
Coordinates:
    height     float64 8B 2.0
  * lat        (lat) float64 216B 34.32 35.72 37.12 38.52 ... 67.94 69.34 70.74
    lat_bnds   (lat, bnds) float64 432B dask.array<chunksize=(27, 2), meta=np.ndarray>
  * lon        (lon) float64 264B 0.0 1.406 2.812 4.219 ... 42.19 43.59 45.0
    lon_bnds   (lon, bnds) float64 528B dask.array<chunksize=(33, 2), meta=np.ndarray>
  * time       (time) datetime64[ns] 248B 2000-08-01T12:00:00 ... 2000-08-31T...
    time_bnds  (time, bnds) datetime64[ns] 496B dask.array<chunksize=(31, 1), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    tasmax     (time, lat, lon) float32 110kB dask.array<chunksize=(31, 27, 33), meta=np.ndarray>
    tasmin     (time, lat, lon) float32 110kB dask.array<chunksize=(31, 27, 33), meta=np.ndarray>
    pr         (time, lat, lon) float32 110kB dask.array<chunksize=(31, 27, 33), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  32873.0
    cmor_version:           3.4.0
    ...                     ...
    title:                  MIROC6 output prepared for CMIP6
    tracking_id:            hdl:21.14100/a505ef46-85f1-4c45-bdfc-722d81290e6d...
    variable_id:            tasmax
    variant_label:          r4i1p1f1
    netcdf_tracking_ids:    hdl:21.14100/a505ef46-85f1-4c45-bdfc-722d81290e6d...
    version_id:             v20191016

In [20]:
"/".join(cmip.col_subsets[0].df.zstore[0].split('/')[4:-4]+[region])+'/'

'CMIP/MIROC/MIROC6/historical/r4i1p1f1/day/europe/'